In [1]:
#envt:HF
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import time

In [2]:
model_name = "distilbert-base-uncased-distilled-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
from transformers import pipeline 

In [4]:
#!pip install pdfminer

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Error processing line 1 of C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\hatch_autorun_farm_haystack.pth:

  Traceback (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<string>", line 6, in <module>
    File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\haystack\__init__.py", line 18, in <module>
      import pandas as pd
  ModuleNotFoundError: No module named 'pandas'

Remainder of file ignored


In [5]:
dir = "D:/demo/Cyber_laws_India_new.pdf"

In [6]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open(dir, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

op2=output_string.getvalue()
print(op2)

Chapter 19 

 

Cyber Laws in India 

Objectives:  This chapter presents the meaning and definition of cyber crime, the legislation in India 
dealing with offences relating to the use of or concerned with the abuse of computers or other electronic 
gadgets.   The Information Technology Act 2000 and the I.T. Amendment Act 2008 have been dealt 
with in detail and other legislations dealing with electronic offences have been discussed in brief. 

Introduction: 

Crime is both a social and economic phenomenon. It is as old as human society. Many ancient 
books  right  from  pre-historic  days,  and  mythological  stories  have  spoken  about  crimes 
committed  by  individuals  be  it  against  another individual  like  ordinary  theft  and  burglary  or 
against  the  nation  like  spying,  treason  etc.  Kautilya’s  Arthashastra  written  around  350  BC, 
considered  to  be  an  authentic  administrative  treatise  in  India,  discusses  the  various  crimes, 
security initiatives to be

In [7]:
import nltk
from nltk import tokenize
from nltk.tokenize import BlanklineTokenizer
paras=BlanklineTokenizer().tokenize(op2)
paras

['Chapter 19',
 'Cyber Laws in India',
 'Objectives:  This chapter presents the meaning and definition of cyber crime, the legislation in India \ndealing with offences relating to the use of or concerned with the abuse of computers or other electronic \ngadgets.   The Information Technology Act 2000 and the I.T. Amendment Act 2008 have been dealt \nwith in detail and other legislations dealing with electronic offences have been discussed in brief.',
 'Introduction:',
 'Crime is both a social and economic phenomenon. It is as old as human society. Many ancient \nbooks  right  from  pre-historic  days,  and  mythological  stories  have  spoken  about  crimes \ncommitted  by  individuals  be  it  against  another individual  like  ordinary  theft  and  burglary  or \nagainst  the  nation  like  spying,  treason  etc.  Kautilya’s  Arthashastra  written  around  350  BC, \nconsidered  to  be  an  authentic  administrative  treatise  in  India,  discusses  the  various  crimes, \nsecurity in

In [8]:
question_answer_pipeline = pipeline("question-answering", model=model,tokenizer=tokenizer)

In [17]:
def retrieverAndPrintAnswer(q,qa_pipeline,context):
    start_time = time.time()
    max_sim=-1;
    index_sim=-1;
    for index,sentence in enumerate(context):
        result = qa_pipeline(question=q, context = sentence)
        sim = result['score']
        if (sim > max_sim):
            max_sim = sim;
            index_sim = index;
            print(index_sim,sim,result['answer'])
        
    print("\n")
    print("Question: ",q)
    print("\n")
    print("Retrieved: ",context[index_sim])
    print("\n")
    print("Search took {:.2f} seconds".format(time.time() - start_time))
    #if sim > 0.001 :
        #print("Retrieved: ",dtf[index_sim])
    #else:
        #print("Retrieved: Unable to find answer to the given question")
        
    #print(index_sim) 

In [18]:
question=" What is digital signature?"
retrieverAndPrintAnswer(question,question_answer_pipeline,paras)

0 0.47394558787345886 19
1 0.865324854850769 Cyber Laws
57 0.8705646991729736 7
65 0.8861892819404602 66B


Question:   What is digital signature?


Retrieved:  66B  Dishonestly  receiving  stolen  computer  resource or  communication  device  with  punishment  upto 
three years or one lakh rupees as fine or both.


Search took 7.86 seconds


In [19]:
question=" What is digital signature?"
retrieverAndPrintAnswer(question,question_answer_pipeline,paras)

0 0.47394558787345886 19
1 0.865324854850769 Cyber Laws
57 0.8705646991729736 7
65 0.8861892819404602 66B


Question:   What is digital signature?


Retrieved:  66B  Dishonestly  receiving  stolen  computer  resource or  communication  device  with  punishment  upto 
three years or one lakh rupees as fine or both.


Search took 19.98 seconds


In [20]:
q2="what are Objectives of I.T. legislation in India?"
retrieverAndPrintAnswer(q2,question_answer_pipeline,paras)

0 0.4605337083339691 Chapter 19
1 0.7120645642280579 Cyber Laws
3 0.8458550572395325 Introduction
82 0.9319742918014526 Transmission of electronic message and communication
156 0.9411342740058899 information 
security and cyber crime prevention


Question:  what are Objectives of I.T. legislation in India?


Retrieved:  In  the  UK,  the  Data  Protection  Act  and  the  Privacy  and  Electronic  Communications 
Regulations  etc  are  all  regulatory  legislations  already  existing  in  the  area  of  information 
security and cyber crime prevention, besides cyber crime law passed recently in August 2011. 
Similarly, we have cyber crime legislations and other rules and regulations in other nations.


Search took 12.83 seconds


In [21]:
q3="what is Electronic governance issues and procedures and the legal ?"
retrieverAndPrintAnswer(q3,question_answer_pipeline,paras)

0 0.5909977555274963 Chapter 19
1 0.9436318874359131 Cyber Laws


Question:  what is Electronic governance issues and procedures and the legal ?


Retrieved:  Cyber Laws in India


Search took 12.27 seconds


In [22]:
q4="what is serious evil in any developing economy?"
retrieverAndPrintAnswer(q4,question_answer_pipeline,paras)

0 0.7098941206932068 19
1 0.9633676409721375 Cyber Laws
4 0.9670875072479248 Crime
138 0.9870887398719788 cyber crimes


Question:  what is serious evil in any developing economy?


Retrieved:  Let us now discuss some of the other relevant legislations in the nation that deal with cyber crimes in 
various sectors.


Search took 11.23 seconds


In [30]:
result = question_answer_pipeline(question=q2, context=op2)

In [31]:
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}")

Answer: 'enhance  corporate  responsibility,  enhance  financial  disclosures,  and 
combat corporate and accounting fraud', score: 0.3891, start: 70965


In [23]:
res_1 = question_answer_pipeline(question=q2, context=op2)

In [24]:
print(f"Answer: '{res_1['answer']}', score: {round(res_1['score'], 4)}, start: {res_1['start']}")

Answer: 'enhance  corporate  responsibility,  enhance  financial  disclosures,  and 
combat corporate and accounting fraud', score: 0.3891, start: 70965


In [26]:
res_2 = question_answer_pipeline(question=q3, context=op2)

In [27]:
print(q3)
print(f"Answer: '{res_2['answer']}', score: {round(res_2['score'], 4)}, start: {res_2['start']}")

what is Electronic governance issues and procedures and the legal ?
Answer: 'Legal Recognition of Electronic Documents', score: 0.6444, start: 6087


In [28]:
res_3 = question_answer_pipeline(question=q4, context=op2)

In [29]:
print(q4)
print(f"Answer: '{res_3['answer']}', score: {round(res_3['score'], 4)}, start: {res_3['start']}")

what is serious evil in any developing economy?
Answer: 'money laundering', score: 0.6476, start: 66784


In [ ]:
start_time = time.time()
print("Search took {:.2f} seconds".format(time.time() - start_time))